In [1]:
import ollama
import gradio as gr

In [2]:
MODEL = 'llama3.2:latest'

SYSTEM_PROMPT = "You are a highly intelligent, friendly, and helpful AI assistant. " \
"Your goal is to provide accurate, clear, and engaging responses tailored to the user's needs. " \
"Answer concisely but thoroughly, ensuring responses are informative and well-structured. " \
"Adapt your tone based on the user’s style—be professional when needed and casual when appropriate. " \
"Prioritize clarity, empathy, and usefulness. If you don’t know something, admit it rather than making assumptions. " \
"Be proactive in asking clarifying questions when necessary. Maintain a positive and respectful tone at all times."

In [15]:
class AssistantOllama:
    def __init__(self, model,system_prompt):
        self.system_prompt = system_prompt
        self.model = model
        self.history = []

    def chat(self, message, history=None): 
        print(history)
        if self.history != []:
            messages = [{"role": "system", "content": SYSTEM_PROMPT}] + self.history + [{"role": "user", "content": message}]
        else:
            messages = [{"role": "system", "content": SYSTEM_PROMPT}] + [{"role": "user", "content": message}]
        
        # Get response from Ollama
        response = ollama.chat(model=self.model, messages=messages)
        
        # Append user message and assistant response
        self.history.append({"role": "user", "content": message})
        self.history.append({"role": "assistant", "content": response.message.content})
        print(response.message.content)
        return response.message.content

    def stream(self, message, history=None):
        print(history)
        if self.history != []:
            messages = [{"role": "system", "content": SYSTEM_PROMPT}] + self.history + [{"role": "user", "content": message}]
        else:
            messages = [{"role": "system", "content": SYSTEM_PROMPT}] + [{"role": "user", "content": message}]
        
        # Start the streaming response
        stream = ollama.chat(
            model=MODEL,
            messages=messages,
            stream=True  # Make sure stream parameter is set to True
        )
        
        # Stream each chunk as it arrives
        response = ""
        for chunk in stream:
            if 'message' in chunk and chunk['message'].get('content'):
                new_text = chunk['message']['content']
                response += new_text
                yield response  # With Gradio streaming, we yield the full response each time
                
        # Append user message and full assistant response to history
        self.history.append({"role": "user", "content": message})
        self.history.append({"role": "assistant", "content": response})

    def clear(self):
        self.history = []
        

In [16]:
assistant = AssistantOllama(model='llama3.2:latest', system_prompt=SYSTEM_PROMPT)

In [8]:
print(assistant.history)

[]


In [ ]:
# Create the Gradio interface
chat_ui = gr.Interface(
    fn=assistant.chat,  # Function to process input
    inputs="text",      # User input type
    outputs="text",     # Output type
    title="Simple Chatbot"
)

# Launch the UI
chat_ui.launch()

In [ ]:
# Create the Gradio interface
with gr.Blocks(css="footer {visibility: hidden}") as chatbot_interface:
    gr.Markdown("# Simple Chatbot")
    gr.Markdown("Welcome! Type a message and press Enter to chat.")
    
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Type your message here...", container=False)
    clear = gr.Button("Clear conversation")
    
    msg.submit(
        assistant.chat,
        [msg, chatbot],
        [chatbot],
        api_name="chat"
    ).then(
        lambda: "",
        None,
        [msg],
        queue=False
    )
    
    clear.click(assistant.clear, None, chatbot)

chatbot_interface.launch()

In [14]:
assistant.history

[{'role': 'user', 'content': 'test'},
 {'role': 'assistant',
  'content': "It looks like you're testing the waters!\n\nIs there anything specific you'd like to talk about or ask? I'm here to help with any questions, provide information, or just have a friendly conversation. Let me know how I can assist you!"},
 {'role': 'user', 'content': 'test 2'},
 {'role': 'assistant',
  'content': "You're testing again!\n\nIt seems like you're just making sure everything is working smoothly. I appreciate the extra confirmation.\n\nIf you're ready to move on, feel free to ask me anything or start a new conversation. If not, we can keep playing around and see how far we can take it!"},
 {'role': 'user', 'content': 'what is the capital of paris?'},
 {'role': 'assistant',
  'content': "The capital of Paris is actually... Paris itself!\n\nParis is a city, not a country, so it doesn't have a traditional capital. The city is the capital of France, a country located in Europe.\n\nHowever, if you're asking 

[{'role': 'user', 'metadata': None, 'content': 'more information', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'd be happy to provide more information about Paris and France.\n\n**Paris**\n\n* Population: approximately 2.1 million people within the city limits\n* Known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, the Louvre Museum, and many more\n* World-famous art scene, fashion capital, and culinary destination\n* Rich history dating back to the Roman era, with numerous historical sites and museums\n\n**France**\n\n* Official language: French (also widely spoken in Quebec, Canada)\n* Capital city: Paris (as we discussed earlier)\n* Area: 643,801 km² (248,573 sq mi)\n* Population: approximately 67 million people\n* Major cities: Marseille, Lyon, Toulouse, Nice, Bordeaux\n\n**Cultural Significance**\n\nFrance is known for its cultural influences, including:\n\n* Art and Architecture: Impressionism, Surrealism, and Art Nouveau movem

In [12]:
with gr.Blocks() as demo:
    chat = gr.ChatInterface(assistant.stream, type="messages")
    clear_button = gr.Button("Clear Chat")

    # When the button is clicked, clear chat and run extra function
    clear_button.click(lambda: ([], assistant.clear()), outputs=chat)

demo.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [ ]:
gr.ChatInterface(fn=assistant.chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


[]
It looks like you're testing the waters!

Is there anything specific you'd like to talk about or ask? I'm here to help with any questions, provide information, or just have a friendly conversation. Let me know how I can assist you!


In [17]:
gr.ChatInterface(fn=assistant.stream, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': None, 'content': 'test', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "It looks like you're testing to see how I respond! Don't worry, I'm designed to handle tests and provide accurate answers.\n\nTo confirm, would you like me to:\n\nA) Engage in a conversation on a specific topic\nB) Ask you a question or request clarification on something\nC) Practice providing responses to different types of questions\nD) Something else (please specify)\n\nLet me know how I can assist you!", 'options': None}]
[{'role': 'user', 'metadata': None, 'content': 'test', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "It looks like you're testing to see how I respond! Don't worry, I'm designed to handle tests and provide accurate answers.\n\nTo confirm, would you like me to:\n\nA) Engage in a conversation on a specific topic\nB) Ask you a question or request clarification on something\nC) Practice providing responses to different

In [ ]:
def stream_ollama(prompt, history=None):
    if history != None:
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": prompt}]
    else:
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + [{"role": "user", "content": prompt}]
    
    # Start the streaming response
    stream = ollama.chat(
        model=MODEL,
        messages=messages,
        stream=True  # Make sure stream parameter is set to True
    )
    
    # Stream each chunk as it arrives
    response = ""
    for chunk in stream:
        if 'message' in chunk and chunk['message'].get('content'):
            new_text = chunk['message']['content']
            response += new_text
            yield response  # With Gradio streaming, we yield the full response each time

In [ ]:
view = gr.Interface(
    fn=assistant.stream,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()  # You might want share=True for testing